In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_2882965/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer()
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_2_4_4,0.919836,-0.175257,0.914247,0.783325,0.088718,0.137819,0.058098,0.100303,"Hidden Size=[15, 7], regularizer=0.1, learning..."
1,model_2_4_0,0.917437,0.235709,0.926265,0.846716,0.091373,0.089626,0.049955,0.070958,"Hidden Size=[15, 7], regularizer=0.1, learning..."
2,model_2_4_1,0.913816,-0.151154,0.923263,0.792767,0.095380,0.134992,0.051990,0.095932,"Hidden Size=[15, 7], regularizer=0.1, learning..."
21,model_2_1_1,0.886453,0.871398,0.892994,0.887186,0.125663,0.087273,0.134597,0.109543,"Hidden Size=[15, 7], regularizer=0.1, learning..."
22,model_2_1_2,0.885060,0.864048,0.818009,0.838755,0.127204,0.092260,0.228916,0.156569,"Hidden Size=[15, 7], regularizer=0.1, learning..."
34,model_2_1_0,0.882258,0.854978,0.924409,0.900261,0.130305,0.098415,0.095082,0.096847,"Hidden Size=[15, 7], regularizer=0.1, learning..."
35,model_2_3_7,0.881666,-0.039620,0.881656,0.821015,0.130961,0.089176,0.129521,0.108162,"Hidden Size=[15, 7], regularizer=0.1, learning..."
40,model_2_3_6,0.880421,-0.067479,0.881732,0.818985,0.132339,0.091566,0.129439,0.109388,"Hidden Size=[15, 7], regularizer=0.1, learning..."
42,model_2_3_5,0.877421,-0.135799,0.882127,0.814188,0.135659,0.097426,0.129006,0.112287,"Hidden Size=[15, 7], regularizer=0.1, learning..."
43,model_2_0_13,0.877311,0.841497,0.388821,0.748386,0.135780,0.160475,0.230105,0.193242,"Hidden Size=[15, 7], regularizer=0.1, learning..."
